In [1054]:
import os
import requests
import shutil
import re
import pandas as pd
from random import randrange
from datetime import datetime
import numpy as np

In [1055]:
codes = ['Extreme', 'Heavy', 'Serious', 'Light', 'Hardcore', 'Rated R', 'violent', 'Rated XXX', 'M/f', 'M/f+', 'M+/f', 'M+/f+', 'F/f', 'F/f+', 'F+/f', 'F+/f+', 'M/m', 'M/m+', 'M+/m', 'M+/m+', 'F/m', 'F/m+', 'F+/m', 'F+/m+', 'm-self', 'f-self', 'MF/f', 'M/mf', 'F/mf', 'MF/m', 'MF/mf', 'F+/fm', 'bestiality', 'cbt', 'enema', 'exhibition', 'fisting', 'incest', 'interracial', 'lactation', 'modification', 'Young', 'pregnant', 'scatology', 'size', 'snuff', 'spanking', 'teen', 'toys', 'transgender', 'transvestite', 'voyeurism', 'WaterSport', 'Swallowing', 'Tit Torture', 'D/s', 'B/D', 'S/M', 'BDSM', 'feet', 'hair', 'latex', 'slavery', 'lingerie', 'bondage', 'chastity', 'belt', 'Rape', 'bible', 'college', 'gothic', 'historical', 'HighSchool', 'job', 'real', 'ScFi', 'slow', 'spoof', 'Fantasy', 'documentary', 'On Line', 'romantic', 'consensual', 'reluctant', 'mc', 'humiliation', 'blackmail', 'torture', 'nc', 'Forced']





In [1056]:
gay_codes = ['M/m', 'M/m+', 'M+/m', 'M+/m+', 'm-self', 'M/mf', 'MF/m', 'MF/mf']

In [1057]:
male_dom_codes = ['M/f', 'M/f+', 'M+/f', 'M+/f+', 'MF/f']

In [1058]:
femdom_codes = ['F/f','F/f+','F+/f','F+/f+','F/m','F/m+','F+/m','F+/m+','MF/f','F/mf','F+/fm']

In [1059]:
male_femdom_codes = ['F/m','F/m+','F+/m','F+/m+','F/mf','F+/fm']

In [1060]:
set(codes) & set(femdom_codes)

{'F+/f',
 'F+/f+',
 'F+/fm',
 'F+/m',
 'F+/m+',
 'F/f',
 'F/f+',
 'F/m',
 'F/m+',
 'F/mf',
 'MF/f'}

In [1090]:
important_codes = ['slavery', 'nc']

In [1062]:
def delete_files(folder = 'stories'):
    shutil.rmtree(folder)
    os.makedirs(folder)

In [1063]:
delete_files()
delete_files('fav_stories')

In [1064]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [1066]:
#delete_files('stories')

In [1067]:
def save_text(text, name, path):
    html_output_name = path+'/' + name
    with open(html_output_name, 'w') as f:
        f.write(text)
        f.close()

In [1068]:
def cleanhtml(raw_html):
    CLEANR = re.compile('<.*?>') 
    cleantext = re.sub(CLEANR, '', raw_html)
    cleantext = cleantext[cleantext.find('CDATA'):]
    cleantext = cleantext[cleantext.find('BDSM Library'):]
    cleantext = cleantext[:cleantext.find('AC_FL_RunContent')]
    cleantext = remove_tags(cleantext)
    return cleantext

In [1069]:
def save_story(story_id, folder = 'stories'):
    url1 = f'https://www.bdsmlibrary.com/stories/story.php?storyid={story_id}'
    url2 = f'https://www.bdsmlibrary.com/stories/wholestory.php?storyid={story_id}'
    req = requests.get(url2, 'html.parser')
    
    text = cleanhtml(req.text)
    
    save_text(text, 'story' + str(story_id)+'.txt', folder)

In [1070]:
fav_story_ids = [3160, 5357, 496, 10501, 9171, 491, 1241, 851, 10344, 24, 886]
#3160 Cheerleaders' Rape and Torture
#5357 From Society Girl To Slave
#496 Dear Diary
#10501 Sara the Puppy Girl
#9171 Lesbian Slave Auction II: Return to Stonebriar
#491 Stacy's Senior Year 
#1241 Two for Torment
#851 Teacher's Ransom
#10344 European Nightmare
#24 Captured Caroline
#886 Lady Jane Greystone's Remarkable Experiment 

In [1071]:
important_codes = ['slavery']

In [1072]:
for i in fav_story_ids:
    save_story(i)

In [1073]:
def clean_codes(line):
    line = line.replace(' ', '').replace('\n', '')
    line = line[line.find(':')+1:]
    CLEANR = re.compile('<.*?>') 
    cleantext = re.sub(CLEANR, ' ', line)
    return cleantext.split()

In [1074]:
def full_id(id):
    return '0'*(5-len(str(id)))+str(id)
full_id(34)

'00034'

In [1121]:
class Story():
    def __init__(self, id):
        self.id = id
        url1 = f'https://www.bdsmlibrary.com/stories/story.php?storyid={self.id}'
        page1 = requests.get(url1, 'html.parser').text
        
        url2 = f'https://www.bdsmlibrary.com/stories/wholestory.php?storyid={self.id}'
        self.link = url2
        url2_text = requests.get(url2, 'html.parser').text
        self.title = url2_text[url2_text.find('title')+21:url2_text.find('</titl')]
        self.content = cleanhtml(url2_text)
        self.length = len(self.content)
        if len(self.title) > 200:
            return None       
        rating_text = page1[page1.find('/10,')-10:page1.find('/10,')+20]
        self.votes = int(rating_text[rating_text.find(',')+2:rating_text.find('votes')-1])
        self.rating = float(rating_text[rating_text.find('(')+1:rating_text.find('/')])
        self.total_readers = page1[page1.find('Total')+6:page1.find('readers')-1]
        k = page1.find('stories/author.php?authorid=')+28
        self.author_id = page1[k:page1.find('">', k)]
        self.author = page1[page1.find('>',k)+1:page1.find('<',k)]
        added_on = page1[page1.find('Added on')+14:page1.find('</', page1.find('Added on')+14)]
        self.added_on = added_on
        
        self.story_codes = clean_codes(page1[page1.find('Story Codes'):page1.find('Synopsis',page1.find('Story Codes')+1)])
        self.synopsis = page1[page1.find('Synopsis:')+14:page1.find('\n',page1.find('Synopsis:')+14)].strip()
        for code in important_codes:
            setattr(self, code, code in self.story_codes)
        self.gay = len(set(self.story_codes) & set(gay_codes)) > 0
        self.femdom = len(set(self.story_codes) & set(femdom_codes)) > 0
        self.male_dom = len(set(self.story_codes) & set(male_dom_codes)) > 0
        self.only_femdom = self.femdom & (self.male_dom == 0) & (self.gay == 0)
        
    def save(self, folder = 'stories'):
        save_text(self.content, full_id(self.id)+' ' + self.title.replace('/', ' ')+'.txt', folder)
        print(f'story {self.id} saved')

In [1124]:
s = Story(3000)

In [1125]:
s.synopsis

"The school faculty have had it with Tami's antics, each teacher taking revenge for the girl's disobedience...each in their own special way."

In [1112]:
len('Synopsis:</b>\n')

14

In [1078]:
s.only_femdom

False

In [1079]:
set(s.story_codes)

{'D/s', 'Light', 'MF/f', 'nc', 'reluctant', 'slavery'}

In [1080]:
set(femdom_codes)

{'F+/f',
 'F+/f+',
 'F+/fm',
 'F+/m',
 'F+/m+',
 'F/f',
 'F/f+',
 'F/m',
 'F/m+',
 'F/mf',
 'MF/f'}

In [1081]:
len(set(s.story_codes) & set(femdom_codes))

1

In [1132]:
def save_stories(ids_list= np.arange(10060,11000,1), folder_name = 'stories', delete = False):
    if delete == True:
        # начинаем с 0
        delete_files(folder_name)
        d = {'story_id': [], 'story_name': [], 'synopsis': [], 'length': [], 'rating':[],'n_votes':[],'total_readers':[],'author_id':[],'author':[],'added_on':[], 'codes':[]}
        for code in important_codes:
            d[code] = []
        df = pd.DataFrame(d)
        df.to_csv(folder_name+'.csv', index=False)
    else:
        df = pd.read_csv(folder_name+'.csv')
    i = 0    
    while i < len(ids_list):
        story_id = ids_list[i]
        
        saved_stories = list(pd.Series(os.listdir(folder_name)).apply(lambda x: x[0:5]))
        if full_id(story_id) in saved_stories:
            i+=1
            continue
        
        
        s = Story(story_id)
        i +=1
        if len(s.title) < 200:
            l = [s.id, s.title, s.synopsis, s.length, s.rating, s.votes, s.total_readers, s.author_id, s.author, s.added_on, str(s.story_codes)]
            df.loc[df.shape[0]] = l+[getattr(s, code) for code in important_codes]
            s.save(folder_name)
            df.to_csv(folder_name+'.csv', index=False)

In [1133]:
save_stories(delete = False)

story 10070 saved
story 10071 saved
story 10072 saved
story 10073 saved
story 10074 saved
story 10075 saved
story 10076 saved
story 10077 saved
story 10078 saved
story 10079 saved
story 10080 saved
story 10081 saved
story 10082 saved
story 10083 saved
story 10084 saved
story 10085 saved
story 10086 saved
story 10087 saved
story 10088 saved
story 10089 saved
story 10090 saved
story 10091 saved
story 10092 saved
story 10093 saved
story 10094 saved
story 10095 saved
story 10096 saved
story 10097 saved
story 10098 saved
story 10099 saved
story 10100 saved
story 10101 saved
story 10102 saved
story 10103 saved
story 10104 saved
story 10105 saved
story 10106 saved
story 10107 saved
story 10108 saved
story 10109 saved
story 10110 saved
story 10111 saved
story 10112 saved
story 10113 saved
story 10114 saved
story 10115 saved
story 10116 saved
story 10117 saved
story 10118 saved
story 10119 saved
story 10120 saved
story 10121 saved
story 10122 saved
story 10123 saved
story 10124 saved
story 1012

story 10528 saved
story 10529 saved
story 10530 saved
story 10531 saved
story 10532 saved
story 10533 saved
story 10534 saved
story 10535 saved
story 10536 saved
story 10537 saved
story 10538 saved
story 10539 saved
story 10540 saved
story 10541 saved
story 10542 saved
story 10543 saved
story 10544 saved
story 10545 saved
story 10546 saved
story 10547 saved
story 10548 saved
story 10549 saved
story 10550 saved
story 10551 saved
story 10552 saved
story 10553 saved
story 10554 saved
story 10555 saved
story 10556 saved
story 10557 saved
story 10558 saved
story 10559 saved
story 10560 saved
story 10561 saved
story 10562 saved
story 10563 saved
story 10564 saved
story 10565 saved
story 10566 saved
story 10567 saved
story 10568 saved
story 10569 saved
story 10570 saved
story 10571 saved
story 10572 saved
story 10573 saved
story 10574 saved
story 10575 saved
story 10576 saved
story 10577 saved
story 10578 saved
story 10579 saved
story 10580 saved
story 10581 saved
story 10582 saved
story 1058

story 10985 saved
story 10986 saved
story 10987 saved
story 10988 saved
story 10989 saved
story 10990 saved
story 10991 saved
story 10992 saved
story 10993 saved
story 10994 saved


In [1129]:
df = pd.read_csv('stories.csv')

In [1130]:
df

,story_id,story_name,synopsis,length,rating,n_votes,total_readers,author_id,author,added_on,codes,slavery,nc
0,1.0,Adventures In Rubber,"Let's talk about your fetish of rubber, high h...",305611.0,9.0,7.0,90742,117,"Bill ""Gomez"" Lemieux","Jun 25, 2000","['M/f', 'B/D', 'latex', 'Serious']",0.0,0.0
1,2.0,Pooch's Story,The miserable slave was punished by her master...,497310.0,8.0,15.0,143848,2,Mr. Ed,"Jun 27, 2000","['M/f', 'F/f', 'scatology', 'B/D', 'S/M', 'tor...",0.0,0.0
2,3.0,Owned Teacher,A female teacher was controlled by her student...,438448.0,9.0,102.0,769300,3,thumb,"Jun 27, 2000","['F/f', 'slavery', 'lingerie', 'humiliation', ...",1.0,0.0
3,4.0,The Sarcophagus,She was sexually tortured and forced to reach ...,170678.0,9.0,22.0,123959,4,JG Leathers,"Jun 29, 2000","['M/f', 'bondage', 'Serious']",0.0,0.0


In [1096]:
df

,story_id,story_name,length,rating,n_votes,total_readers,author_id,author,added_on,codes,slavery,nc
0,1.0,Adventures In Rubber,305611.0,9.0,7.0,90741,117,"Bill ""Gomez"" Lemieux","Jun 25, 2000","['M/f', 'B/D', 'latex', 'Serious']",0.0,0.0
1,2.0,Pooch's Story,497310.0,8.0,15.0,143847,2,Mr. Ed,"Jun 27, 2000","['M/f', 'F/f', 'scatology', 'B/D', 'S/M', 'tor...",0.0,0.0
2,3.0,Owned Teacher,438448.0,9.0,102.0,769296,3,thumb,"Jun 27, 2000","['F/f', 'slavery', 'lingerie', 'humiliation', ...",1.0,0.0
3,4.0,The Sarcophagus,170678.0,9.0,22.0,123958,4,JG Leathers,"Jun 29, 2000","['M/f', 'bondage', 'Serious']",0.0,0.0
4,5.0,White victim,130401.0,9.5,8.0,93906,5,Allene Blake,"Jun 29, 2000","['M/f', 'F/f', 'F+/f', 'B/D', 'nc', 'Serious']",0.0,1.0
5,6.0,Diane,152608.0,7.0,6.0,159462,6,SD,"Jun 29, 2000","['F/f', 'WaterSport', 'slavery', 'bondage', 'h...",1.0,1.0


In [1038]:
def str_to_list(s):
    return s.replace('[', '').replace(']', '').replace(',', '').replace("'", '').split()

In [1042]:
str_to_list(df.codes[21])

['M/f', 'F/f', 'S/M', 'humiliation', 'Serious']

In [1046]:
df['femdom_male'] = ~df.gay.apply(bool)&df.femdom.apply(bool) & df.codes.apply(lambda x: len(set(str_to_list(x))&set(male_femdom_codes))>0)

In [1053]:
df[df.femdom_male][df.nc == 1].sort_values(by = 'n_votes', ascending = False).head(20)

/var/folders/1_/mhjdmxtx6kl7blgbrm54g4zr0000gn/T/ipykernel_15247/944822470.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.femdom_male][df.nc == 1].sort_values(by = 'n_votes', ascending = False).head(20)


,story_id,story_name,length,rating,n_votes,total_readers,author_id,author,added_on,codes,slavery,nc,femdom,gay,male_dom,only_femdom,femdom_male
3116,3178.0,The Femina Empire,199172.0,7.0,20.0,133143,1269.0,Pluto,"Feb 1, 2005","['F/m', 'F+/m+', 'exhibition', 'slavery', 'bon...",1.0,1.0,1.0,0.0,0.0,1.0,True
1375,1411.0,Gentle Steps to Complete Domination,102048.0,9.0,18.0,152264,606.0,Andrew Devine,"Dec 30, 2002","['F/m', 'cbt', 'enema', 'exhibition', 'spankin...",0.0,1.0,1.0,0.0,0.0,1.0,True
456,466.0,The Prisoner,2101932.0,9.5,15.0,639270,236.0,Will Cane,"Aug 20, 2001","['F+/f+', 'F+/m', 'cbt', 'exhibition', 'spanki...",0.0,1.0,1.0,0.0,0.0,1.0,True
1383,1419.0,A Game of Torture,51408.0,8.0,14.0,76173,567.0,BDSM Bill,"Jan 4, 2003","['F+/m', 'MF/f', 'cbt', 'spanking', 'BDSM', 'r...",0.0,1.0,1.0,0.0,1.0,0.0,True
1485,1525.0,The Best Laid Plans,551636.0,8.0,13.0,340534,581.0,Lex Ludite,"Mar 7, 2003","['M/f', 'M+/f', 'M+/f+', 'F/f', 'F+/f', 'F+/m'...",0.0,1.0,1.0,0.0,1.0,0.0,True
666,682.0,Sunder,533269.0,9.5,12.0,117657,30.0,Unknown,"Jan 16, 2002","['M/f', 'F/f', 'F+/f', 'F+/f+', 'F/m', 'f-self...",1.0,1.0,1.0,0.0,1.0,0.0,True
2902,2963.0,The Stepford Husbands,57603.0,9.5,11.0,69710,306.0,Jane Marwood,"Nov 14, 2004","['F+/m', 'F+/m+', 'enema', 'exhibition', 'Youn...",0.0,1.0,1.0,0.0,0.0,1.0,True
2308,2364.0,Mom's Mistake,532453.0,7.5,11.0,446650,973.0,Jethro Jodhpur,"Mar 18, 2004","['M/f', 'M+/f', 'F+/f', 'F/m', 'MF/f', 'bestia...",0.0,1.0,1.0,0.0,1.0,0.0,True
859,884.0,A Modern Human Dairy and Meat Farm,64372.0,3.5,11.0,83559,406.0,Pick Up Man,"Apr 16, 2002","['M/f', 'F/f', 'F+/f', 'F+/m', 'cbt', 'exhibit...",1.0,1.0,1.0,0.0,1.0,0.0,True
2998,3060.0,Dexter Chronicles,386296.0,9.0,10.0,153876,1221.0,Mysty Mason,"Dec 18, 2004","['F/m', 'incest', 'reluctant', 'nc', 'Heavy']",0.0,1.0,1.0,0.0,0.0,1.0,True


In [1004]:
def filter_stories(df):
    data = df[df.rating >= 9]
    data = data[data.gay == 0]
    data = data[data.n_votes >=3]
    data = data[data.nc + data.slavery >=1]
    return data

In [1005]:
data = filter_stories(df)

In [1016]:
data.head()

,story_id,story_name,length,rating,n_votes,total_readers,author_id,author,added_on,codes,slavery,nc,femdom,gay,male_dom,only_femdom
2,3.0,Owned Teacher,438448.0,9.0,102.0,769048,3.0,thumb,"Jun 27, 2000","['F/f', 'slavery', 'lingerie', 'humiliation', ...",1.0,0.0,1.0,0.0,0.0,1.0
4,5.0,White victim,130401.0,9.5,8.0,93871,5.0,Allene Blake,"Jun 29, 2000","['M/f', 'F/f', 'F+/f', 'B/D', 'nc', 'Serious']",0.0,1.0,1.0,0.0,1.0,0.0
12,13.0,Janet in Training,703802.0,9.5,25.0,116688,14.0,sfmaster,"Jun 30, 2000","['F/f', 'B/D', 'S/M', 'slavery', 'consensual',...",1.0,0.0,1.0,0.0,0.0,1.0
13,14.0,The Challenge,783036.0,9.5,17.0,119177,14.0,sfmaster,"Jun 30, 2000","['F/f', 'F/mf', 'B/D', 'S/M', 'slavery', 'cons...",1.0,0.0,1.0,0.0,0.0,1.0
18,19.0,Lynne,1328585.0,9.0,44.0,359758,16.0,Just Another Bloke,"Jun 30, 2000","['M/f', 'F/f', 'B/D', 'S/M', 'slavery', 'bonda...",1.0,0.0,1.0,0.0,1.0,0.0


In [976]:
df[df.rating >= 8]

,story_id,story_name,length,rating,n_votes,total_readers,author_id,author,added_on,codes,slavery,nc,femdom,gay,male_dom,only_femdom
0,1.0,Adventures In Rubber,305611.0,9.0,7.0,90689,117,"Bill ""Gomez"" Lemieux","Jun 25, 2000","['M/f', 'B/D', 'latex', 'Serious']",0.0,0.0,0.0,0.0,1.0,0.0
1,2.0,Pooch's Story,497310.0,8.0,15.0,143780,2,Mr. Ed,"Jun 27, 2000","['M/f', 'F/f', 'scatology', 'B/D', 'S/M', 'tor...",0.0,0.0,1.0,0.0,1.0,0.0
2,3.0,Owned Teacher,438448.0,9.0,102.0,769048,3,thumb,"Jun 27, 2000","['F/f', 'slavery', 'lingerie', 'humiliation', ...",1.0,0.0,1.0,0.0,0.0,1.0
3,4.0,The Sarcophagus,170678.0,9.0,22.0,123903,4,JG Leathers,"Jun 29, 2000","['M/f', 'bondage', 'Serious']",0.0,0.0,0.0,0.0,1.0,0.0
4,5.0,White victim,130401.0,9.5,8.0,93871,5,Allene Blake,"Jun 29, 2000","['M/f', 'F/f', 'F+/f', 'B/D', 'nc', 'Serious']",0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,1953.0,Changing Hands,32551.0,9.5,4.0,9280,816,Fidelis Blue,"Sep 20, 2003","['M+/f', 'B/D', 'real', 'consensual', 'Light']",0.0,0.0,0.0,0.0,1.0,0.0
1907,1954.0,Raping Trisha,66926.0,8.5,8.0,93132,831,Alebeard,"Sep 20, 2003","['M/f+', 'spanking', 'teen', 'S/M', 'humiliati...",0.0,1.0,0.0,0.0,1.0,0.0
1908,1955.0,The Novice Master,15597.0,9.0,3.0,12167,816,Fidelis Blue,"Sep 24, 2003","['M+/f', 'spanking', 'B/D', 'bondage', 'real',...",0.0,0.0,0.0,0.0,1.0,0.0
1909,1956.0,A Proper Lesson,17226.0,9.5,2.0,14841,816,Fidelis Blue,"Sep 24, 2003","['M/f', 'spanking', 'D/s', 'consensual', 'Seri...",0.0,0.0,0.0,0.0,1.0,0.0


In [992]:
df[df.rating >=9][df.n_votes >=3]

/var/folders/1_/mhjdmxtx6kl7blgbrm54g4zr0000gn/T/ipykernel_15247/3244950602.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.rating >=9][df.n_votes >=3]


,story_id,story_name,length,rating,n_votes,total_readers,author_id,author,added_on,codes,slavery,nc,femdom,gay,male_dom,only_femdom
0,1.0,Adventures In Rubber,305611.0,9.0,7.0,90689,117,"Bill ""Gomez"" Lemieux","Jun 25, 2000","['M/f', 'B/D', 'latex', 'Serious']",0.0,0.0,0.0,0.0,1.0,0.0
2,3.0,Owned Teacher,438448.0,9.0,102.0,769048,3,thumb,"Jun 27, 2000","['F/f', 'slavery', 'lingerie', 'humiliation', ...",1.0,0.0,1.0,0.0,0.0,1.0
3,4.0,The Sarcophagus,170678.0,9.0,22.0,123903,4,JG Leathers,"Jun 29, 2000","['M/f', 'bondage', 'Serious']",0.0,0.0,0.0,0.0,1.0,0.0
4,5.0,White victim,130401.0,9.5,8.0,93871,5,Allene Blake,"Jun 29, 2000","['M/f', 'F/f', 'F+/f', 'B/D', 'nc', 'Serious']",0.0,1.0,1.0,0.0,1.0,0.0
6,7.0,Dee's Tao - The Way Of An Anatomically Challen...,310100.0,9.0,8.0,78165,7,Andante,"Jun 29, 2000","['F/f', 'D/s', 'S/M', 'consensual', 'humiliati...",0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,2836.0,The Big Tease,57878.0,9.0,5.0,52692,1153,Tarrachus,"Sep 10, 2004","['MF/f', 'toys', 'BDSM', 'consensual', 'tortur...",0.0,0.0,1.0,0.0,1.0,0.0
2779,2839.0,Role Playing III - Raven and the Inquisitor,63374.0,9.0,5.0,21697,385,sarijak,"Sep 11, 2004","['F/f', 'f-self', 'bondage', 'romantic', 'tort...",0.0,0.0,1.0,0.0,0.0,1.0
2780,2840.0,Becky's Slave,338305.0,9.5,11.0,230771,630,Doug Young,"Sep 15, 2004",['F/m'],0.0,0.0,1.0,0.0,0.0,1.0
2783,2843.0,The Afternoon that Devastated my Life,75756.0,9.0,3.0,28278,1155,Sean Dunne,"Sep 15, 2004","['F/m', 'cbt', 'spanking', 'BDSM', 'latex', 'b...",0.0,0.0,1.0,0.0,0.0,1.0


In [820]:
fav_story_ids = [3160, 5357, 496, 10501, 9171, 491, 1241, 851, 10344, 24, 886]
#24 Captured Caroline
#491 Stacy's Senior Year 
#496 Dear Diary
#851 Teacher's Ransom
#886 Lady Jane Greystone's Remarkable Experiment
#1241 Two for Torment
#3160 Cheerleaders' Rape and Torture
#5357 From Society Girl To Slave
#9171 Lesbian Slave Auction II: Return to Stonebriar
#10344 European Nightmare
#10501 Sara the Puppy Girl

In [821]:
fav_story_ids

[3160, 5357, 496, 10501, 9171, 491, 1241, 851, 10344, 24, 886]

In [822]:
d = {'story_id': [], 'story_name': [], 'rating':[],'n_votes':[],'total_readers':[],'author_id':[],'author':[],'added_on':[], 'codes':[]}
for code in important_codes:
    d[code] = []
df = pd.DataFrame(d)
df.to_csv('fav_stories.csv', index=False)

In [823]:
delete_files('fav_stories')
for story_id in fav_story_ids:
    s = Story(story_id)
    story_id +=1
    if len(s.title) < 200:
        l = [s.id, s.title, s.rating, s.votes, s.total_readers, s.author_id, s.author, s.added_on, str(s.story_codes)]
        df.loc[df.shape[0]] = l+[getattr(s, code) for code in important_codes]
        s.save('fav_stories')
        df.to_csv('fav_stories.csv', index=False)

In [824]:
df = pd.read_csv('fav_stories.csv')

In [825]:
df['total'] = df.slavery + df.nc

In [826]:
df

,story_id,story_name,rating,n_votes,total_readers,author_id,author,added_on,codes,slavery,nc,torture,Serious,violent,humiliation,B/D,bestiality,D/s,Forced,total
0,3160.0,Cheerleaders' Rape and Torture,9.5,66.0,1775714,369,Semiater,"Jan 29, 2005","['M+/f+', 'bestiality', 'enema', 'fisting', 's...",1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0
1,5357.0,From Society Girl To Slave,8.0,9.0,96636,364,Anonymous,"Mar 27, 2007","['F+/f', 'enema', 'modification', 'toys', 'D/s...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0
2,496.0,Parker 01: Dear Diary,9.5,7.0,35070,251,Parker,"Sep 17, 2001","['MF/f', 'D/s', 'slavery', 'reluctant', 'nc', ...",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
3,10501.0,Sara the Puppy Girl,8.5,9.0,225870,4124,Lokker12,"Jun 22, 2014","['M/f', 'enema', 'modification', 'teen', 'toys...",1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
4,9171.0,Lesbian Slave Auction II: Return to Stonebriar,7.5,8.0,255050,856,Jlou,"May 8, 2011","['F+/f+', 'incest', 'teen', 'TitTorture', 'BDS...",1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
5,491.0,Stacy's Senior Year,9.0,17.0,132275,251,Parker,"Sep 9, 2001","['MF/mf', 'teen', 'slavery', 'HighSchool', 'sl...",1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
6,1241.0,Two for Torment,10.0,8.0,236702,550,pamela,"Oct 13, 2002","['M/f', 'incest', 'B/D', 'S/M', 'torture', 'nc...",0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
7,851.0,Parker 21: Teacher's Ransom,9.0,4.0,51839,251,Parker,"Apr 1, 2002","['M/f', 'F/f', 'slavery', 'nc', 'Serious']",1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
8,10344.0,European Nightmare,9.0,3.0,145763,4051,JoshJones,"Sep 15, 2013","['M/f', 'M/f+', 'M+/f', 'M+/f+', 'F/f', 'F/f+'...",0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
9,24.0,Captured Caroline,9.5,28.0,190581,20,Quin,"Jul 7, 2000","['M/f', 'B/D', 'nc', 'Serious']",0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [742]:
df.codes

0     ['M+/f+', 'bestiality', 'enema', 'fisting', 's...
1     ['F+/f', 'enema', 'modification', 'toys', 'D/s...
2     ['MF/f', 'D/s', 'slavery', 'reluctant', 'nc', ...
3     ['M/f', 'enema', 'modification', 'teen', 'toys...
4     ['F+/f+', 'incest', 'teen', 'TitTorture', 'BDS...
5     ['MF/mf', 'teen', 'slavery', 'HighSchool', 'sl...
6     ['M/f', 'incest', 'B/D', 'S/M', 'torture', 'nc...
7            ['M/f', 'F/f', 'slavery', 'nc', 'Serious']
8     ['M/f', 'M/f+', 'M+/f', 'M+/f+', 'F/f', 'F/f+'...
9                       ['M/f', 'B/D', 'nc', 'Serious']
10    ['M+/f', 'bestiality', 'slavery', 'historical'...
Name: codes, dtype: object

In [771]:
l = []
for i in range(11):
    print(df.story_name[i],df.codes[i])

Cheerleaders' Rape and Torture ['M+/f+', 'bestiality', 'enema', 'fisting', 'spanking', 'teen', 'toys', 'WaterSport', 'BDSM', 'slavery', 'bondage', 'real', 'humiliation', 'torture', 'nc', 'Extreme']
From Society Girl To Slave ['F+/f', 'enema', 'modification', 'toys', 'D/s', 'B/D', 'BDSM', 'feet', 'latex', 'slavery', 'lingerie', 'bondage', 'chastitybelt', 'reluctant', 'humiliation', 'blackmail', 'torture', 'nc', 'Extreme', 'Serious', 'violent']
Parker 01: Dear Diary ['MF/f', 'D/s', 'slavery', 'reluctant', 'nc', 'Light']
Sara the Puppy Girl ['M/f', 'enema', 'modification', 'teen', 'toys', 'Swallowing', 'TitTorture', 'B/D', 'S/M', 'BDSM', 'latex', 'slavery', 'bondage', 'real', 'humiliation', 'torture', 'Forced', 'Heavy', 'violent']
Lesbian Slave Auction II:  Return to Stonebriar ['F+/f+', 'incest', 'teen', 'TitTorture', 'BDSM', 'latex', 'slavery', 'lingerie', 'bondage', 'chastitybelt', 'Rape', 'torture', 'nc', 'Forced', 'Extreme']
Stacy's Senior Year ['MF/mf', 'teen', 'slavery', 'HighSchoo

In [755]:
df.codes[0]+df.codes[1]

"['M+/f+', 'bestiality', 'enema', 'fisting', 'spanking', 'teen', 'toys', 'WaterSport', 'BDSM', 'slavery', 'bondage', 'real', 'humiliation', 'torture', 'nc', 'Extreme']['F+/f', 'enema', 'modification', 'toys', 'D/s', 'B/D', 'BDSM', 'feet', 'latex', 'slavery', 'lingerie', 'bondage', 'chastitybelt', 'reluctant', 'humiliation', 'blackmail', 'torture', 'nc', 'Extreme', 'Serious', 'violent']"